In [1]:
import os

# # Set the proxy environment variables
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import datetime
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [5]:
ee.Initialize()

In [6]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../.')

# import the temporal_consistency_check class
from BackGround_modules.Class_5_Temporal_consistency_check import Temporal_consistency_check
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

In [7]:
# define basic parameters
past_years = [f'{i}_{i+2}'for i in range(1990,2020,3)]
future_years = ['2020_2022']
year_range = past_years + future_years

# print out the year_ranges
year_range

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019',
 '2020_2022']

In [8]:
# all regions
region_cns = ['华东',   '东北',   '中南',    '华北',  '西北', '西南']
region_ens = ['huadong','dongbei','zhongnan','huabei','xibei','xinan']

# region for analysis
region_cn = '西南'
region_en = 'xinan'

# region shp for exporting
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
            .filterMetadata('NAME1','equals',region_cn)

In [9]:
# import COPERNICUS.urban_coverfraction layer, which will be used to masking urban classfications
COPERNICUS_urban = ee.Image(f'COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select("urban-coverfraction")
China_LUCC = ee.Image("users/wangjinzhulala/Paper_3/01_LUCC_1990_2019_china/CLCD_v01_2019").eq(8)

past_img = ee.Image("users/wangjinzhulala/China_built_up/06_temporal_corrected_classification/Mosaic_only_forward_China_huadong_sample_ensemble").gt(0)
past_img_buffer = past_img.reduceNeighborhood(
                              reducer='mean',
                              kernel=ee.Kernel.circle(3)
                            ).gt(0)

# other masking layers
dem = ee.Image("USGS/SRTMGL1_003")
slope = ee.Terrain.slope(dem)
precipitation = ee.Image("users/wangjinzhulala/China_built_up/07_supplementary_data/China_precipication_2010s")


# create the mask_rule_dict
mask_rule = {'dongbei':precipitation.lt(550),
             'huabei':slope.gt(4),
             'huadong':slope.gt(4),
             'xibei':dem.gt(1000).Or(precipitation.lt(500)),
             'xinan':ee.Image(1),
             'zhongnan':slope.gt(4)}

# function to perform the masking
def mask_img(in_img,region_name=region_en):
    
    # 1) first clip out the area to be masked by COPERNICUS_urban
    img_mask_area = in_img.updateMask(mask_rule[region_name])
    img_mask_area = img_mask_area.multiply(past_img_buffer)
    
    # 2) then clip out the area remain unchanged
    img_unchange_area = in_img.updateMask(mask_rule[region_name].Not())
    
    # 3) mosaic the two imgs
    merge_img = ee.ImageCollection([img_mask_area,img_unchange_area]).mosaic()
    
    
    return merge_img

In [10]:
# define the threshold to create ten_fold_check image out of ten random_sample_splited layers
cross_fold_threshold = 4

# path to fetch/save result
sample_path  = 'users/wangjinzhulala/China_built_up/04_sample_train_test_split'
img_path     = 'users/wangjinzhulala/China_built_up/05_primary_classification'
img_out_path = 'users/wangjinzhulala/China_built_up/06_temporal_corrected_classification'

# the historical temporal_checked_img
past_checked_img = \
 ee.Image("users/wangjinzhulala/China_built_up/06_temporal_corrected_classification/Mosaic_only_forward_China_huadong_sample_ensemble")\
 .clip(Region)

past_each_year_img = {year:ee.Image(past_checked_img).gt(idx) for idx,year in enumerate(past_years[::-1])}

### Classification tricks


| [Image]   | [mask]                                           |
|:---------|:-------------------------------------------------|
| dongbei  | {precipitation_2010s<550} & {urban_percent>20}   |
| huabei   | {slope>4} & {urban_percent>20}                   |
| huadong  | {slope>4} & {urban_percent>20}                   |
| xibei    | {dem>(1000) or (precipitation<500) or (slope>4)} & {urban_percent>20}                   |
| xinan    | {dem>3000} & {urban_percent>20}                  |
| zhongnan | {slope>4} & {urban_percent>20}                   |


##### Splipt the classification imgs into two part:
- before 2014: do not mask 
- after 2014: do mask

#### If the masking was from start (i.e, 1990-1992)

In [11]:
# masked_imgs = []

# for year in year_range:
    
#     # all classification imgs with different seeds
#     imgs = [ee.Image(f"{img_path}/Spectrum_Normalize_Fourier_Terrain_Meterology_{region_en}_{year}_{seed}") 
#             for seed in range(10)]
    
#     # sum these classifications and the img>4 will be the final result
#     ten_folds_check = ee.ImageCollection(imgs).sum().gt(cross_fold_threshold)
    
#     # masking: from the classfication tricks
#     if int(year[:4]) > 2013:
#         # !!!! change mask_img accordingly !!!!
#         ten_folds_check = mask_img(ten_folds_check)
    
#     # append masked imgs to list
#     masked_imgs.append(ten_folds_check)

#### If the masking was from middle (i.e., 2020-2022)

In [12]:
# Something wrong here!!!!!!

In [13]:
masked_imgs = list(past_each_year_img.values())[::-1]

# first fetch the masked imgs from past_classifications
for year in future_years:
    
    # all classification imgs with different seeds
    imgs = [ee.Image(f"{img_path}/Spectrum_Normalize_Fourier_Terrain_Meterology_{region_en}_{year}_{seed}") 
            for seed in range(10)]
    
    # sum these classifications and the img>4 will be the final result
    ten_folds_check = ee.ImageCollection(imgs).sum().gt(cross_fold_threshold).rename("remapped")
    
    # masking: from the classfication tricks
    if int(year[:4]) > 2013:
        # !!!! change mask_img accordingly !!!!
        ten_folds_check = mask_img(ten_folds_check).gt(0).rename("remapped")
    
    # append masked imgs to list
    masked_imgs.append(ten_folds_check)

In [ ]:
# add the img to map
Map = geemap.Map()
Map.centerObject(Region,10)
Map.add_basemap('HYBRID')

# Map.addLayer(masked_imgs[0], {'min':0,'max':1},'Iter_1')
# Map.addLayer(masked_imgs[2], {'min':0,'max':1},'Iter_3')
# Map.addLayer(masked_imgs[4], {'min':0,'max':1},'Iter_5')
# Map.addLayer(masked_imgs[6], {'min':0,'max':1},'Iter_7')
# Map.addLayer(masked_imgs[8], {'min':0,'max':1},'Iter_9')
Map.addLayer(masked_imgs[9], {'min':0,'max':1},'Iter_10')
Map.addLayer(masked_imgs[-1], {'min':0,'max':1},'Iter_11')

Map

### Perform temporal consistency check

In [ ]:
# get the checked results
Iteration_num = 10
Window_size = [2,3,4,5,6]
modes = ['only_forward'] #['backward_forward','forward_backward','only_forward','only_backward']

Check_df_list = []

for window in Window_size:
    for mode in modes:
    
        # get the checked classification imgs, here they are stored in a dictionary
        Iter_temporal_check_instaces = Temporal_consistency_check(masked_imgs,window,Iteration_num)\
                                        .Iterate_the_check(mode = mode)

        # convert the Iter_temporal_check_instaces to a Dataframe
        Check_instances = pd.DataFrame(Iter_temporal_check_instaces).T
        Check_instances.columns = year_range

        # Add the window-size and iteration-num to the df
        Check_instances['Mode'] = [mode] * Iteration_num
        Check_instances['Window'] = [window] * Iteration_num
        Check_instances['Iteration'] = range(1,Iteration_num+1)

        # Reorder the columns
        cols = Check_instances.columns
        Check_instances = Check_instances[list(cols[-3:]) + list(cols[:-3])] 

        # add the checked df into list
        Check_df_list.append(Check_instances)

In [ ]:
# merge all instace_df together
Check_df = pd.concat(Check_df_list)
Check_df.reset_index(inplace=True,drop=True)

##### Transform the Check_sample into long format so each row is an observation with spicific window-iteration-year condition

In [ ]:
# attach Check_sample to the Check_instaces_long 
Check_instaces_long = pd.DataFrame(Check_df.set_index(['Mode','Window','Iteration']).stack()).reset_index()
Check_instaces_long.columns = ['Mode','Window','Iteration','year','Image']

In [ ]:
Check_instaces_long

##### Create the Mosaic image from all temporal-checked imgs

In [ ]:
Mosaic_img = {}

for window in Window_size:
    for iteration in range(1,Iteration_num+1):
        # get the temporal_checked imgs
        Temporal_checked_select  = Check_instaces_long[(Check_instaces_long['Iteration'] == iteration)&\
                                                       (Check_instaces_long['Window']    == window)     ]['Image'].values
#         # reclassify the classification img
#         Temporal_checked_select_re = [img.remap([0,1],[0,len(Temporal_checked_select)-i]) 
#                                       for i,img in enumerate(Temporal_checked_select)]

#         # mosaic all checked imgs together
#         Classification_Mosaic = ee.ImageCollection(Temporal_checked_select_re).max()

        Classification_Mosaic = ee.Image(0)
        for img in Temporal_checked_select:   
            Classification_Mosaic = Classification_Mosaic.add(img)

        
        # store the img to dict
        Mosaic_img[(window,iteration)] = [Classification_Mosaic]

In [ ]:
# convert the mosaic classification into a dataframe and format the column names
Mosaic_df = pd.DataFrame(Mosaic_img).T

# format the column names
Mosaic_df_reindex = Mosaic_df.reset_index()
Mosaic_df_reindex.columns = ['window','iteration','mosaic']

##### Add the checked img to the map

In [ ]:
# Color_ramp for Mosaic img
Mosaic_VIS = {'min':0,
              'max':len(masked_imgs),
              "palette":["000000","3288bd","66c2a5","abdda4",
                         "e6f598","ffffbf","fee08b","fdae61",
                         "f46d43","d53e4f","9e0142"]}

In [ ]:
# define filter parameters
mode = 'only_forward' # 'backward_forward' |'forward_backward'|'only_forward'|'only_backward'
window_len = 3
iterantion_num = 9
year_idx = '1990_1992'


# get the temporal checked imgs
Temporal_checkd = Check_instaces_long[(Check_instaces_long['Mode']   == mode)&
                                      (Check_instaces_long['Window'] == window_len)&
                                      (Check_instaces_long['year']   == year_idx)]

# get the checked img with different iteration
img_year_window_filtered = [Temporal_checkd[Temporal_checkd['Iteration'] == i]['Image'].values[0] 
                            for i in range(1,Iteration_num + 1)]

# get the 
Mosaic_image = Mosaic_df_reindex[(Mosaic_df_reindex['window'] == window_len) \
                               &(Mosaic_df_reindex['iteration'] == iterantion_num)]['mosaic'].values[0]

In [ ]:
# add the img to map
Map = geemap.Map()
Map.centerObject(Region,10)
Map.add_basemap('HYBRID')

# create a color ramp for mosaiced img
Mosaic_VIS = {"opacity":1,'min':0,"max":10,
              "palette":["000000","3288bd","66c2a5","abdda4","e6f598",
                         "ffffbf","fee08b","fdae61","f46d43","d53e4f","9e0142"]}

# add image to map

# Map.addLayer(img_year_window_filtered[0], {'min':0,'max':1},'Iter_1')
# Map.addLayer(img_year_window_filtered[2], {'min':0,'max':1},'Iter_3')
# Map.addLayer(img_year_window_filtered[4], {'min':0,'max':1},'Iter_5')
# Map.addLayer(img_year_window_filtered[6], {'min':0,'max':1},'Iter_7')
# Map.addLayer(img_year_window_filtered[8], {'min':0,'max':1},'Iter_9')
# Map.addLayer(img_year_window_filtered[9], {'min':0,'max':1},'Iter_10')

Map.addLayer(Mosaic_image,                  Mosaic_VIS,       'Mosaic')

Map

##### Export the temporal checked classifications

Export each year's classification img of temporal_checked

In [ ]:
# here we find that temporcal checked img with 
# "Mode=forward_backward", "Iteration=9", "Window=3"
# are the best quality imgs

Target_temporal_checked = Check_instaces_long[(Check_instaces_long['Mode']      == 'only_forward')&\
                                              (Check_instaces_long['Iteration'] == 9)&\
                                              (Check_instaces_long['Window']    == 3)]

Target_mosaci_img = Mosaic_df_reindex[(Mosaic_df_reindex['window'] == 3) \
                                     &(Mosaic_df_reindex['iteration'] == 9)]['mosaic'].values[0].toInt8()

Export the mosaiced classification img 

In [ ]:
# Export the result
export_name = f'Mosaic_only_forward_{region_en}'


#export to GEE assset
task = ee.batch.Export.image.toAsset(    
                                        image= Classification_Mosaic,
                                        description=export_name,
                                        assetId=f'{img_out_path}/{export_name}',
                                        region=Region.geometry().bounds(),
                                        scale=30,
                                        maxPixels=int(1e13)
)

# # to cloud_storage
# task = ee.batch.Export.image.toCloudStorage(image = Target_mosaci_img,
#                                             description=export_name,
#                                             bucket='north_china_plain',
#                                             fileNamePrefix=f'China_built/{export_name}',
#                                             region=Region.geometry().bounds(),
#                                             scale=30,
#                                             maxPixels=int(1e13),
#                                             skipEmptyTiles=True)

task.start()
    

### Assess the accuracy after the temporal check

In [30]:
# import the untouched control points
Check_sample = [ee.FeatureCollection(f'{sample_path}/Grid_select_{region_en}_{year}') for year in year_range]


# Sample_Sentinel = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Sentinel_2017_2019")\
#                     .randomColumn('split', 101)\
#                     .filterMetadata('split','less_than',0.25)

# # create sample list for accuracy assesment
# Check_sample = [Sample_Landsat] * len_landsat_1990_2013 + \
#                [Sample_Sentinel] * len_sentinel_2014_2019

In [31]:
# create a dictionary stores are year-value pairs
Mosaic_value = dict(zip(year_range,range(Iteration_num,0,-1)))


# calculate the accuracy using the untouched control samples
Accuracy_dict = {}

# double loop to calculate the accuracy
for idx,row in Mosaic_df_reindex.iterrows():
    
    window     = row[0]
    iteration  = row[1]
    mosaic_img = row[2]

    for i,year in enumerate(year_range):

        # get the img and untouched sample
        sample = Check_sample[i]

        # get the classificaiton, rename the band so we can use Accuracy_assesment module
        img_my   = ee.Image(mosaic_img.gte(Mosaic_value[year])).rename('classification')

        # extract the img pixel value to test_sample
        test_with_My   = img_my.sampleRegions(  collection = sample, 
                                                properties = ['Built'], 
                                                scale      = 30,
                                                tileScale = 6)


        # compute the accuracy and put them into a df
        Accuracy_dict[(window,iteration,year)] = test_with_My.errorMatrix('Built','classification')

KeyError: '2020_2022'

In [ ]:
# compute the accuracy from GEE server in chunck
chunck_size = 10

# get all accuracy instances
error_matrix_instances = [v for k,v in Accuracy_dict.items()]

# calculate the accuracy by chunck
acc_value = []

# set flags to report the process
flag = 0 
flag_total = len(error_matrix_instances)/chunck_size

# loop through the chunk to get accuracy
for i in range(0,len(error_matrix_instances),chunck_size):
    
    chunck = error_matrix_instances[i:i+chunck_size]
    acc = ee.List(chunck).map(lambda mat:ee.ConfusionMatrix(mat).accuracy()).getInfo()
    acc_value.extend(acc)
    
    # print out the process
    flag = flag + 1 
    print(f'The operation of {flag}/{int(flag_total)} is complete!')

In [ ]:
# sotre the accuracy into a dataframe and formate thedf
acc_k = [k for k,v in Accuracy_dict.items()]
acc_df = pd.DataFrame(dict( (k,[v *100]) for k,v in zip(acc_k,acc_value))).T

# format the df
Checked_acc = acc_df.reset_index()
Checked_acc.columns = ['window','iteration','year','accuracy']

##### Compute the accuracy after temporal-check

In [ ]:
# Save the accuracy to disk
Checked_acc.to_csv(f'./Result/Temporal_check_acc_{region_en}.csv',index=False)

# load the acc_df
Checked_acc = pd.read_csv(f'./Result/Temporal_check_acc_{region_en}.csv')

# concert these columns into category, so we can make figures
Checked_acc['iteration'] = Checked_acc['iteration'].astype('category')
Checked_acc['year'] = Checked_acc['year'].astype('category')

In [ ]:
Checked_acc

##### Observe that window-size of 3 is the optimun size, because the bigger the window size, the more imgs wouldn't be temporal corrected. So we choose 3 as window size even though 5 look like the best one

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data = Checked_acc,x='window',y='accuracy')

##### Observe that  9 iterations  achieved a stable accuracy. here we neglect the seemingly high accuracy at lower iterations because we care more about stable performance, which is the key of temporal correction

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data = Checked_acc[Checked_acc['window'] == 3],
             x='iteration',
             y='accuracy')

##### Observe that the temporal correction have incresed the classification accuracy

1) import the original accuracy df

In [ ]:
# access the original acc_df
Sentinel_Acc = pd.read_csv('../Process_1_GEE_Python_Classification/Sub_Process_7_Classification_on_img/Result/Classification_Accuracy_landsat_sentinel.csv')
Landsat_Acc  = pd.read_csv('../Process_1_GEE_Python_Classification/Sub_Process_7_Classification_on_img/Result/Classification_Accuracy.csv')

In [ ]:
# Define the classification only use landsat as input
landsat_col = ['Landsat_1990_1992', 'Landsat_1993_1995', 'Landsat_1996_1998','Landsat_1999_2001',
               'Landsat_2002_2004', 'Landsat_2005_2007','Landsat_2008_2010', 'Landsat_2011_2013']

# concatenate all accuracy into one df
Original_landsat = Landsat_Acc[Landsat_Acc['year'].isin(landsat_col)]
Original_acc = pd.concat([Original_landsat,Sentinel_Acc])

In [ ]:
# formeting the original addcuracy_df
Original_acc['Year'] = Original_acc['year'].apply(lambda x: x[-9:])
Original_acc

2) get the 10-folds threshold correction

In [ ]:
# read data
Threshold_acc_df = pd.read_csv('../Process_1_GEE_Python_Classification/Sub_Process_8_Determine_the_threshold_for_the_sum_of_10_random_classification/Result/Threshold_acc_df.csv')

# make the threshold categorical so we can make figures
Threshold_acc_df['Threshold'] = Threshold_acc_df['Threshold'].astype('category')
Threshold_acc_df['Year'] = Threshold_acc_df['Year'].astype('category')

3) make a figure to compare all the accuracy 

In [ ]:
plt.figure(figsize=(12,8))

# original accuracy
sns.lineplot(data = Original_acc,x = 'Year',y='Overall_ACC',label = 'Original')


# threshold correction
sns.lineplot(data = Threshold_acc_df[Threshold_acc_df['Threshold']==4],
             x='Year',
             y='Accuracy',
            label = 'Threshold Correction')


# temporal correction
sns.lineplot(data = Checked_acc[((Checked_acc['window']==3)&
                                 (Checked_acc['iteration']==9))],
             x='year',
             y='accuracy',
            label = 'Temporal Correction')